# Tensor Compression

In this example, we implement the optimization of a tensor networked based on the algorithm by Pollman et al. [1]. This finds the optimal 2-qubit tensors required to approximate an update based on an Ising Hamiltonian.

In [1]:
using Qaintensor
using LinearAlgebra
using StatsBase: sample
using Qaintessent


Below, we generate the Ising hamiltonian with a transverse field, given a timestep of $\delta t  = 0.1$

\begin{equation}
H_{ising} = \sum_{<i,j>}{\sigma_{x}^{(i)}\sigma_{x}^{(j)}} + h\sum_{j}{\sigma_{z}^{(j)}}
\end{equation}
The notation $<i,j>$ indicates that sum runs over adjacent sites on a given lattice.

In [2]:
N = 7

function rkron(m, N::Int)
    for i in 1:N-1
        m = kron(m, Matrix{ComplexF64}(I, 2, 2))
    end
    m
end

function lkron(m, N::Int)
    for i in 1:N
        m = kron(Matrix{ComplexF64}(I, 2, 2), m)
    end
    m
end

function ising_hamiltonian(h::Float64, N::Int)
    N >= 2 || error("Only produces Hamiltionians for N >= 2")
    x_matrix = Qaintessent.matrix(X)
    z_matrix = Qaintessent.matrix(Z)
    
    m = zeros(ComplexF64, (2^N, 2^N))
    hx_matrix = h*x_matrix
    for i in 0:N-1
        m_new = lkron(rkron(hx_matrix, N-i), i)
        m += m_new
    end
    kron_x_matrix = kron(x_matrix, x_matrix)
    for i in 0:N-2
        m_new = lkron(rkron(kron_x_matrix, N-i-1), i)
        m -= m_new
    end
    m
end 
Δt = 0.1
h = rand(Float64)
m = MPO(exp(-im*Δt*ising_hamiltonian(h, N)));

We generate a ground state of $|0_{N}\rangle$ to begin the optimization and find the exact state after 1 timestep, $t_1$.

In [3]:
v = zeros(ComplexF64, 2^N)
v[1] = 1
t0 = MPS(v)
sol = contract(apply_MPO(t0, m, Tuple(1:N)))[:];
t1 = MPS(sol);

We define a function that generates a trotterized layer of 2-dimensional 4x4 tensors initialized to a I state.

In [4]:
function trotterized_layer!(tn::MPS, N)
    wires = length(tn.openidx)
    m = Tensor(reshape(Matrix{ComplexF64}(I, 4, 4), (2,2,2,2)))
    for j in 0:N-1
        for i in 1:wires-1
            push!(tn.tensors, deepcopy(m))
        end
        for i in 1:wires-wires%2
            push!(tn.contractions, Summation([tn.openidx[i], j*(wires-1)+wires+i-(i+1)%2=>2-i%2]))
            tn.openidx[i] = j*(wires-1)+wires+i-(i+1)%2=>4-i%2
        end
        for i in 2:wires-(wires+1)%2
            push!(tn.contractions, Summation([tn.openidx[i], j*(wires-1)+wires+i-i%2=>1+i%2]))
            tn.openidx[i] =  j*(wires-1)+wires+i-i%2=>3+i%2
        end
    end
end

trotterized_layer! (generic function with 1 method)

We set the number of layers $L$ to 2 and apply $L$ layers to the initial state $t_0$.

In [5]:
L = 2

t0_tilde = deepcopy(t0)
trotterized_layer!(t0_tilde, L)

We then define the optimize_layers function, which provides an update step based on the polar decomposition. Refer to [1] for more details.

In [6]:
function combine_tensors(tn0, tn1, wires, shift)
    ev = deepcopy(tn0)
    append!(ev.tensors, tn1.tensors)
    for i in 1:wires
        push!(ev.contractions, Summation([ev.openidx[1], Qaintensor.shift_pair(tn1.openidx[i], shift)]))
        popfirst!(ev.openidx)
    end
    append!(ev.contractions, [Summation(Qaintensor.shift_pair.(m.idx, shift)) for m in tn1.contractions])
    ev
end

function optimize_layers!(tn0, tn1)
    wires = length(tn0.openidx)
    shift = length(tn0.tensors)
    ev = combine_tensors(tn0, tn1, wires, shift)
    tensor_num = length(ev.tensors)
    total_contractions = deepcopy(ev.contractions)
    total_tensors = deepcopy(ev.tensors)
    er=0
    for i in wires+1:tensor_num-wires
        contractions = Summation[]
        open_wires = Pair[]
        for contraction in total_contractions
            bonds = first.(contraction.idx) .== i
            if any(bonds)
                open_wire = [wire.first > i ? wire.first-1=>wire.second : wire for wire in contraction.idx[.!bonds]]
                append!(open_wires, open_wire)
            else
                s = Summation([m.first > i ? m.first-1=>m.second : m for m in contraction.idx])
                push!(contractions, s)
            end
        end
        ev.tensors = append!(total_tensors[1:i-1], total_tensors[i+1:end])
        ev.contractions = contractions
        ev.openidx = open_wires
        output = permutedims(contract(ev), (3,4,1,2))
        U,S,V = svd(reshape(output, (4,4)))
        output = reshape(V*adjoint(U), (2,2,2,2))
        total_tensors[i] = Tensor(output)
        tn0.tensors[i] = total_tensors[i]
        ev.tensors = total_tensors
        ev.contractions = total_contractions
        ev.openidx = []
        er = contract(ev)
    end
    1-norm(er)^2
end

optimize_layers! (generic function with 1 method)

In [7]:
er = Inf
while er > 0.0005
    er = optimize_layers!(t0_tilde, t1);
    println("The error of is: " * string(er))
end

The error of is: 0.00505025600241038
The error of is: 0.00014330334110945753


[1] Lin, Sheng-Hsuan, et al. “Real- and Imaginary-Time Evolution with Compressed Quantum Circuits.” ArXiv:2008.10322 [Cond-Mat, Physics:Quant-Ph], Sept. 2020. arXiv.org, http://arxiv.org/abs/2008.10322.
